# Dijkprofile Annotator - Simple Version
Use this notebook to do the dijkprofile annotation if you don't want to use the `generateAnnotations.py` script.
All you need to do is change the file paths below to point to the correct source csv's and give a correct output path.

In [6]:
# paramaters
surfacelines_path = 'data/dijkprofiles_fryslan/2-2/surfacelines.csv'
characteristicpoints_path = 'data/dijkprofiles_fryslan/2-2/characteristicpoints.csv'
output_path = "data/new_characteristicpoints.csv"

In [7]:
# imports
from __future__ import print_function, division
import numpy as np
import torch.nn.functional as F
import csv
import torch
import torch.nn as nn
import time
import pickle
from sklearn.preprocessing import StandardScaler
import os
import torch
import pandas as pd
import glob
import joblib
import seaborn as sns
import datetime 
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from operator import itemgetter
from torch.autograd import Variable
from PIL import Image

# import functions and globals from dijknet_functions
from dijknet_functions import class_dict, inverse_class_dict
from dijknet_functions import Dijknet, convert_tool_data, ffill, make_annotations

## Load new data

In [8]:
# path to the surfacelines and characteristic points files.
annotation_tuples = [
    (surfacelines_path, characteristicpoints_path)
]

profile_dict_test, surfaceline_dict_test = convert_tool_data(annotation_tuples)

# scale the new profiles.
scaler = joblib.load("pickles/scaler.pik") 
for i, key in enumerate(profile_dict_test.keys()):
    profile_dict_test[key]['profile'] = scaler.transform(profile_dict_test[key]['profile'].reshape(-1,1)).reshape(-1)
    
print("done!")

loaded surfacelines for 166 locations
loaded characteristic points for 166 locations
done!


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

PATH = "models/dijknet_95p.pt"

# construct network
model = Dijknet(1,len(set(class_dict.values())))

# load trained weights into network
model.load_state_dict(torch.load(PATH))

# set network to inference/eval mode
model.eval()

# copy network to device
model.to(device)
print("loaded model!")

loaded model!


# make annotation

In [10]:
make_annotations(profile_dict_test, surfaceline_dict_test, model, output_path, device)

total profiles to predict:  166
done!
